In [1]:
import os, time
import datetime as dt
import pandas as pd
from multiprocessing import Pool, cpu_count
import Settings
SOURCE = Settings.NETFLOW_PROS

In [2]:
from tqdm.notebook import tqdm
from tqdm.notebook import tnrange

In [3]:
INTERVAL = 5
if __name__ == '__main__':
    conv_files = list(filter(lambda x: x.startswith('feather_'), os.listdir(SOURCE)))
    print('[INFO] Starting conversion')
    print(f'[INFO] {len(conv_files)} files found in directory {os.getcwd()}{SOURCE[1:]}')
    df = pd.read_feather(SOURCE+'/'+conv_files[0])
    df['Duration'] = df['Duration'].map(lambda x: dt.timedelta(seconds=x))
    df['Datetime_lastseen'] = df['Datetime_firstseen']+df['Duration']
    if True:
        dfs=[df]
        for file in tqdm(conv_files[1:]):
            tempdf = pd.read_feather(SOURCE+'/'+file)
            tempdf['Duration'] = tempdf['Duration'].map(lambda x: dt.timedelta(seconds=x))
            tempdf['Datetime_lastseen'] = tempdf['Datetime_firstseen']+df['Duration']
            dfs.append(tempdf)
        df = pd.concat(dfs, ignore_index=True)
#     df.sort_values(by=['Datetime_firstseen'], inplace=True)
#     df.reset_index(inplace=True, drop=True)

#     pool = Pool(processes=4)
#     dfs = pool.map(process_nfdump, conv_files)

[INFO] Starting conversion
[INFO] 7 files found in directory D:\Users\Geovanni\Sync\PhD\Data\Conv


In [4]:
display(df.shape)
df

(9496587, 14)

,Datetime_firstseen,Duration,Protocol,Src_IP_Addr,Src_Port,Direc,Dst_IP_Addr,Dst_Port,Flags,Tos,Packets,Bytes,Flows,Datetime_lastseen
0,2020-02-07 23:54:53.840,00:04:50.310000,UDP,134.193.92.68,500,->,209.85.235.189,500,........,0,121,52824,1,2020-02-07 23:59:44.150
1,2020-02-07 23:57:52.820,00:01:51.696000,TCP,172.217.9.132,443,->,134.193.198.19,62831,...AP...,128,2521,1800000,1,2020-02-07 23:59:44.516
2,2020-02-07 23:58:01.226,00:01:43.250000,TCP,172.217.1.238,443,->,134.193.198.19,62833,...AP...,0,1225,990522,1,2020-02-07 23:59:44.476
3,2020-02-07 23:58:34.242,00:01:10.006000,TCP,134.193.254.218,49345,->,150.199.7.126,443,...APRS.,0,715,41665,1,2020-02-07 23:59:44.248
4,2020-02-07 23:58:43.858,00:01:01.010000,TCP,136.34.135.223,51053,->,134.193.136.193,443,...AP.SF,0,16,1832,1,2020-02-07 23:59:44.868
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9496582,2020-02-08 00:34:43.987,00:00:00,TCP,45.227.254.30,49116,->,204.56.1.75,33898,......S.,0,1,40,1,NaT
9496583,2020-02-08 00:34:43.987,00:00:00,TCP,134.49.252.240,58357,->,134.193.25.80,80,......S.,0,1,44,1,NaT
9496584,2020-02-08 00:34:43.987,00:00:00,TCP,92.63.194.115,40352,->,134.193.157.232,35543,......S.,0,1,40,1,NaT
9496585,2020-02-08 00:34:43.987,00:00:00,TCP,198.209.56.129,443,->,134.193.244.195,52995,...AP...,0,1,109,1,NaT


In [5]:
interval_sum = pd.DataFrame(pd.date_range(start=df.Datetime_firstseen.min(), 
                                           end=df.Datetime_lastseen.max(), 
                                           freq=f'{INTERVAL}S'))
interval_sum[1] = [0] * len(interval_sum)
interval_sum.columns = ['datespace', 'sum of data transfer']
print(f'{interval_sum.datespace.min()}\t->\t{interval_sum.datespace.max()}')
interval_sum.set_index('datespace', inplace=True)

# interval_sum = interval_sum.iloc[:int(len(interval_sum)*.25),:]
# interval_sum

2020-02-07 23:30:00.056000	->	2020-02-08 01:04:30.056000


In [6]:
import math
def calculate_resampled_bytes(row, starttime, inter):
    num_interval_transfer = math.ceil(row['Duration'].total_seconds()/inter)
    avg_rate_of_transfer = row['Bytes']/num_interval_transfer
    list_of_bytes_across_intervals = [avg_rate_of_transfer]*num_interval_transfer
    return list_of_bytes_across_intervals

In [ ]:
current_point = 0
stop=False
progress = [x for x in range(0,105,5)]
interval_index_list = list(interval_sum.index)
interval_sum_length = interval_sum.shape[0]
for interval_datetime in tqdm(interval_index_list):
    if interval_index_list.index(interval_datetime)/interval_sum_length >= progress[0]/100:
        print(f'At {interval_datetime}\t{interval_index_list.index(interval_datetime)}\t{progress.pop(0)}%')
    current_interval_index = interval_index_list.index(interval_datetime)
    df_inside_interval = df[(df.Datetime_firstseen >= interval_datetime) & (df.Datetime_firstseen < interval_datetime+dt.timedelta(seconds=INTERVAL))]
#     for ind, row in tqdm(df_inside_interval.iterrows(), total=df_inside_interval.shape[0], leave=False, position=1):
    for ind, row in df_inside_interval.iterrows():
        if row['Datetime_firstseen'] + row['Duration'] >= interval_datetime+dt.timedelta(seconds=5):
            resampled_bytes = calculate_resampled_bytes(row, interval_datetime, INTERVAL)
            for pos, transfer in enumerate(resampled_bytes):
                try:
                    interval_sum.iloc[current_interval_index+pos]+=transfer                    
                except:
                    date_range = pd.date_range(start=interval_sum.index[-1], 
                                               periods=len(resampled_bytes)-pos+1, 
                                               freq=f'{INTERVAL}S').tolist()[1:]
                    interval_sum = pd.concat([interval_sum, pd.DataFrame({'sum of data transfer':resampled_bytes[pos:]}, index=date_range)])
                    break
        else:
            interval_sum.loc[interval_datetime] += row['Bytes']


At 2020-02-07 23:30:00.056000	0	0%

At 2020-02-07 23:34:45.056000	57	5%
At 2020-02-07 23:39:30.056000	114	10%
At 2020-02-07 23:44:15.056000	171	15%
At 2020-02-07 23:48:55.056000	227	20%
At 2020-02-07 23:53:40.056000	284	25%
At 2020-02-07 23:58:25.056000	341	30%

At 2020-02-08 00:03:10.056000	398	35%
At 2020-02-08 00:07:50.056000	454	40%


In [ ]:
print(sum(interval_sum['sum of data transfer']))
print(sum(df['Bytes']))

In [ ]:
interval_sum.iloc[:,:].plot()

In [ ]:
start_interval = '2020-02-08 00:04:40.000'
end_interval = '2020-02-08 00:04:45.056'
df_subset = df[(df['Datetime_firstseen']>=pd.to_datetime(start_interval))&(df['Datetime_firstseen']<pd.to_datetime(end_interval))]

print('Total bytes transfered:\t', end='')
print(sum(df_subset['Bytes']))

print('# of records:\t', end='')
print(df_subset.shape[0])

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

num_intervals = 300
start=100
disp = df.iloc[start:num_intervals+start,:].sample(frac=1)
# disp = df.sample(frac=1).iloc[:num_intervals,:]
intervals = disp[['Datetime_firstseen','Datetime_lastseen','Duration']]

min_date=disp.Datetime_firstseen.min() - dt.timedelta(microseconds=disp.Datetime_firstseen.min().microsecond)
max_date=disp.Datetime_lastseen.max() - dt.timedelta(microseconds=disp.Datetime_firstseen.min().microsecond) + dt.timedelta(seconds=1)


idx=0

viridis = plt.cm.get_cmap('viridis', num_intervals)
              
fig, ax = plt.subplots(figsize=(20,27))
for x_s, dur, x_e in zip(intervals['Datetime_firstseen'],intervals['Duration'], intervals['Datetime_lastseen']):
#     print(x_s, dur.total_seconds(), x_e)
    x_s = (x_s - min_date).total_seconds()
    ax.add_patch(plt.Rectangle((x_s,idx),
                               dur.total_seconds(),
                               .2 ,
                               color=viridis(idx / num_intervals)))
    idx+=1

    
ax.autoscale()

date_range = pd.date_range(start=min_date, end=max_date, periods=len(ax.xaxis.get_majorticklocs()))
date_range = ['{:%b-%d-%Y %H:%M:%S}.{:02.0f}'.format(x, x.microsecond / 10000.0) for x in date_range]
print(date_range)
ax.xaxis.set_ticklabels(date_range)
# ax.set_ylim(-2,4)
plt.savefig('./networkflow')
plt.show()


## Bucket by flow size
## bandwidth
## geolocate ip address into a dicionary (remember load balancing)

In [ ]:
ip_host = pd.read_feather("./feather_hostnames")
ip_host.set_index('ip_addr', inplace=True)

In [ ]:
df['Source_Host'] = df.loc[:,'Src_IP_Addr'].map(lambda x: ip_host.loc[x,'hostnames'])
df['Dest_Host'] = df.loc[:,'Dst_IP_Addr'].map(lambda x: ip_host.loc[x,'hostnames'])
df.head()

In [ ]:
df.head()

In [ ]:
df_umkc = df[df.Dest_Host == 'UMKC']
df_umkc.sort_values(by=['Datetime_firstseen'], inplace=True)
df_umkc.reset_index(drop=True, inplace=True)
df_umkc.head()

In [ ]:
df_umkc.loc[:,'Bytes'].plot()

In [ ]:
df_umkc.loc[600000:900000,'Bytes'].plot()

In [ ]:
df_umkc.loc[600000:900000,'Bytes'].max()